# Day 58

Updating my rankings for teams based on total fantasy points allowed to the position through Week 15. I'll also run it for the past 4 weeks.

In [6]:
import pandas as pd
import sqlite3

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

In [7]:
season = 2022
week = 15

In [8]:
query = f"""
WITH fantasy_totals_by_team AS (
    -- Get the total fantasy points scored per team, per position, per week
    SELECT
        recent_team,
        position,
        week,
        ROUND(SUM(fantasy_points), 2) as tot_pts,
        ROUND(SUM(fantasy_points_ppr), 2) as tot_pts_ppr
    FROM weekly
    WHERE season = {season}
        AND week <= {week}
        AND position in ('QB', 'RB', 'WR', 'TE')
    GROUP BY recent_team, position, week),
-- Get schedule info
    home_games AS (
        SELECT 
            game_id,
            season,
            week, 
            home_team AS team,
            away_team AS opp_team
        FROM schedules
        WHERE season = {season}
            AND week <= {week}
    ),
    away_games AS (
       SELECT 
            game_id,
            season,
            week, 
            away_team AS team,
            home_team AS opp_team
        FROM schedules
        WHERE season = {season}
            AND week <= {week}
    ),
    stacked AS (
        SELECT *
        FROM home_games
        UNION
        SELECT *
        FROM away_games
    ),
-- Join the two temp datasets
-- Every team will have it's total fantasy points per team appended to it
joined_data AS (
    SELECT
        game_id,
        season,
        stacked.week,
        team,
        opp_team,
        position,
        tot_pts,
        tot_pts_ppr
    FROM stacked
    LEFT JOIN fantasy_totals_by_team
        ON fantasy_totals_by_team.recent_team = stacked.team
        AND fantasy_totals_by_team.week = stacked.week),
-- Only use opp_team to get the points scored scored against
total_pts_scored_against AS (
    SELECT
        opp_team,
        position,
        SUM(tot_pts) AS tot_pts_scored_against,
        SUM(tot_pts_ppr) AS tot_pts_ppr_scored_against
    FROM joined_data
    GROUP BY opp_team, position),
-- Utilize RANK() to get the season-level rankings
season_rankings AS (
    SELECT
        opp_team AS team,
        position,
        RANK() OVER(PARTITION BY position ORDER BY tot_pts_scored_against DESC) AS r_pts_scored_against,
        RANK() OVER(PARTITION BY position ORDER BY tot_pts_ppr_scored_against DESC) AS r_pts_ppr_scored_against
    FROM total_pts_scored_against
    ORDER BY position, r_pts_ppr_scored_against)
SELECT *
FROM season_rankings
"""

df = pd.read_sql(query, conn)
df

,team,position,r_pts_scored_against,r_pts_ppr_scored_against
0,DET,QB,1,1
1,MIA,QB,2,2
2,TEN,QB,3,3
3,KC,QB,4,4
4,LV,QB,5,5
...,...,...,...,...
123,GB,WR,28,28
124,HOU,WR,29,29
125,SEA,WR,30,30
126,NYJ,WR,31,31


## Standard Scoring through Week 15

In [9]:
standard = df.drop('r_pts_ppr_scored_against', axis=1).sort_values('r_pts_scored_against')

for pos in ['QB', 'RB', 'WR', 'TE']:
    print(standard.query(f"(position == '{pos}') & (r_pts_scored_against <= 3 | r_pts_scored_against >= 30)"))

   team position  r_pts_scored_against
0   DET       QB                     1
1   MIA       QB                     2
2   TEN       QB                     3
29   SF       QB                    30
30  PHI       QB                    31
31  DEN       QB                    32
   team position  r_pts_scored_against
32  HOU       RB                     1
33  SEA       RB                     2
36  LAC       RB                     3
62   NE       RB                    30
59  TEN       RB                    31
63   SF       RB                    32
    team position  r_pts_scored_against
96   TEN       WR                     1
98   DET       WR                     2
97   MIN       WR                     3
125  SEA       WR                    30
126  NYJ       WR                    31
127  DEN       WR                    32
   team position  r_pts_scored_against
65  SEA       TE                     1
64  ARI       TE                     2
66  MIA       TE                     3
95  WAS       TE  

## PPR Scoring through Week 15

In [10]:
ppr = df.drop('r_pts_scored_against', axis=1).sort_values('r_pts_ppr_scored_against')

for pos in ['QB', 'RB', 'WR', 'TE']:
    print(ppr.query(f"(position == '{pos}') & (r_pts_ppr_scored_against <= 3 | r_pts_ppr_scored_against >= 30)"))

   team position  r_pts_ppr_scored_against
0   DET       QB                         1
1   MIA       QB                         2
2   TEN       QB                         3
29   SF       QB                        30
30  PHI       QB                        31
31  DEN       QB                        32
   team position  r_pts_ppr_scored_against
32  HOU       RB                         1
33  SEA       RB                         2
34   LV       RB                         3
61  DAL       RB                        30
62   NE       RB                        31
63   SF       RB                        32
    team position  r_pts_ppr_scored_against
96   TEN       WR                         1
97   MIN       WR                         2
98   DET       WR                         3
125  SEA       WR                        30
126  NYJ       WR                        31
127  DEN       WR                        32
   team position  r_pts_ppr_scored_against
64  ARI       TE                         1
65  

To get the rankings for just the last 4 weeks I'll update the SQL query slightly, using `BETWEEN` in the `WHERE` clauses.

In [14]:
query = f"""
WITH fantasy_totals_by_team AS (
    -- Get the total fantasy points scored per team, per position, per week
    SELECT
        recent_team,
        position,
        week,
        ROUND(SUM(fantasy_points), 2) as tot_pts,
        ROUND(SUM(fantasy_points_ppr), 2) as tot_pts_ppr
    FROM weekly
    WHERE season = {season}
        AND week BETWEEN {week - 3} AND {week}
        AND position in ('QB', 'RB', 'WR', 'TE')
    GROUP BY recent_team, position, week),
-- Get schedule info
    home_games AS (
        SELECT 
            game_id,
            season,
            week, 
            home_team AS team,
            away_team AS opp_team
        FROM schedules
        WHERE season = {season}
            AND week BETWEEN {week - 3} AND {week}
    ),
    away_games AS (
       SELECT 
            game_id,
            season,
            week, 
            away_team AS team,
            home_team AS opp_team
        FROM schedules
        WHERE season = {season}
            AND week BETWEEN {week - 3} AND {week}
    ),
    stacked AS (
        SELECT *
        FROM home_games
        UNION
        SELECT *
        FROM away_games
    ),
-- Join the two temp datasets
-- Every team will have it's total fantasy points per team appended to it
joined_data AS (
    SELECT
        game_id,
        season,
        stacked.week,
        team,
        opp_team,
        position,
        tot_pts,
        tot_pts_ppr
    FROM stacked
    LEFT JOIN fantasy_totals_by_team
        ON fantasy_totals_by_team.recent_team = stacked.team
        AND fantasy_totals_by_team.week = stacked.week),
-- Only use opp_team to get the points scored scored against
total_pts_scored_against AS (
    SELECT
        opp_team,
        position,
        SUM(tot_pts) AS tot_pts_scored_against,
        SUM(tot_pts_ppr) AS tot_pts_ppr_scored_against
    FROM joined_data
    GROUP BY opp_team, position),
-- Utilize RANK() to get the season-level rankings
season_rankings AS (
    SELECT
        opp_team AS team,
        position,
        RANK() OVER(PARTITION BY position ORDER BY tot_pts_scored_against DESC) AS r_pts_scored_against,
        RANK() OVER(PARTITION BY position ORDER BY tot_pts_ppr_scored_against DESC) AS r_pts_ppr_scored_against
    FROM total_pts_scored_against
    ORDER BY position, r_pts_ppr_scored_against)
SELECT *
FROM season_rankings
"""

df = pd.read_sql(query, conn)
df

,team,position,r_pts_scored_against,r_pts_ppr_scored_against
0,TEN,QB,1,1
1,DET,QB,2,2
2,KC,QB,3,3
3,PHI,QB,4,4
4,JAX,QB,5,5
...,...,...,...,...
123,PIT,WR,26,28
124,NO,WR,29,29
125,WAS,WR,31,30
126,GB,WR,30,31


## Standard Scoring Weeks 13-15

In [15]:
standard = df.drop('r_pts_ppr_scored_against', axis=1).sort_values('r_pts_scored_against')

for pos in ['QB', 'RB', 'WR', 'TE']:
    print(standard.query(f"(position == '{pos}') & (r_pts_scored_against <= 3 | r_pts_scored_against >= 30)"))

   team position  r_pts_scored_against
0   TEN       QB                     1
1   DET       QB                     2
2    KC       QB                     3
29  PIT       QB                    30
30  ATL       QB                    31
31   NO       QB                    32
   team position  r_pts_scored_against
32  SEA       RB                     1
34  IND       RB                     2
33  HOU       RB                     3
62  CAR       RB                    30
63  DET       RB                    31
61   SF       RB                    32
    team position  r_pts_scored_against
98   DAL       WR                     1
96   DET       WR                     2
97   MIN       WR                     3
126   GB       WR                    30
125  WAS       WR                    31
127  ATL       WR                    32
   team position  r_pts_scored_against
66  ATL       TE                     1
65  JAX       TE                     2
64  TEN       TE                     3
94  CHI       TE  

## PPR Scoring Weeks 13-15

In [16]:
ppr = df.drop('r_pts_scored_against', axis=1).sort_values('r_pts_ppr_scored_against')

for pos in ['QB', 'RB', 'WR', 'TE']:
    print(ppr.query(f"(position == '{pos}') & (r_pts_ppr_scored_against <= 3 | r_pts_ppr_scored_against >= 30)"))

   team position  r_pts_ppr_scored_against
0   TEN       QB                         1
1   DET       QB                         2
2    KC       QB                         3
29  PIT       QB                        30
30  ATL       QB                        31
31   NO       QB                        32
   team position  r_pts_ppr_scored_against
32  SEA       RB                         1
33  HOU       RB                         2
34  IND       RB                         3
61   SF       RB                        30
62  CAR       RB                        31
63  DET       RB                        32
    team position  r_pts_ppr_scored_against
96   DET       WR                         1
97   MIN       WR                         2
98   DAL       WR                         3
125  WAS       WR                        30
126   GB       WR                        31
127  ATL       WR                        32
   team position  r_pts_ppr_scored_against
64  TEN       TE                         1
65  